# Federated learning: Simple experiment

In this notebook we provide a simple example of how to make an experiment of a federated environment with the help of this framework. We are going to use a popular dataset to start the experimentation in a federated environment. The framework provides some functions to load the [Emnist](https://www.nist.gov/itl/products-and-services/emnist-dataset) Digits dataset.

In [ ]:
import matplotlib.pyplot as plt
import shfl

database = shfl.data_base.Emnist()
train_data, train_labels, val_data, val_labels, test_data, test_labels = database.load_data()

Let's inspect some properties of the loaded data.

In [ ]:
print(len(train_data))
print(len(val_data))
print(len(test_data))
print(type(train_data[0]))
train_data[0].shape

So, as we have seen, our dataset is composed by a set of matrices of 28 by 28. Before starting with the federated scenario, we can take a look to a sample in the training data.

In [ ]:
plt.imshow(train_data[0])

We are going to simulate a federated learning scenario with a set of client nodes containing private data, and a central server that will be responsible to coordinate the different clients. 
But, first of all, we have to simulate the data contained in every client. 
In order to do that, we are going to use the previous dataset loaded. 
The assumption in this example will be the data is distributed as a set of independent and identically distributed random variables, having every node more o less the same amount of data. 
There are a set of different possibilities in order to distribute the data. 
The distribution of the data is one of the factors that could impact more to a federated algorithm. 
Therefore, the framework contains the implementation of some of the most common distributions that allow you to experiment different situations easily. 
In this [link](https://github.com/sherpaai/Sherpa.FL/tree/master/shfl/data_distribution) you can dig into the options that the framework provides at the moment.

In [ ]:
iid_distribution = shfl.data_distribution.IidDataDistribution(database)
federated_data, test_data, test_labels = iid_distribution.get_federated_data(num_nodes=20, percent=10)

That's it! We have created federated data from the Emnist dataset using 20 nodes and 10 percent of the available data. This data is a set of data nodes containing private data.  Let's learn a little more about the federated data.

In [ ]:
print(type(federated_data))
print(federated_data.num_nodes())
federated_data[0].private_data

As we can see, private data in a node is not accessible directly but the framework provides mechanisms to use this data in a machine learning model. 
A federated learning algorithm is defined by a machine learning model locally deployed in each node that learns from the respective node’s private data and an aggregating mechanism to aggregate the different model parameters uploaded by the client nodes to a central node. 
In this example we will use a deep learning model using keras to build it. The framework provides a class to allow using a Keras model into a federated learning scenario, your job is only to create a function acting as model builder.

In [ ]:
import keras

def model_builder():
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', strides=1, input_shape=(28, 28, 1)))
    model.add(keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(keras.layers.Dropout(0.4))
    model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', strides=1))
    model.add(keras.layers.MaxPooling2D(pool_size=2, strides=2, padding='valid'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.1))
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))

    model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"])
    
    return shfl.model.DeepLearningModel(model)

Now, the only piece missing is the aggregation operator. 
Nevertheless, the framework provides some aggregation operators that we can use. 
In the following piece of code we define the federated aggregation mechanism (see the notebook [Federated learning: Aggregation operators](./basic_concepts_aggregation_operations.ipynb) for more details about aggregation operators, and this [link](https://github.com/sherpaai/Sherpa.FL/tree/master/shfl/federated_aggregator) for their implementation).
Moreover, the class `FederatedGovernment` manages the deployment of the model over the federated data, and is in charge of aggregating the parameters into a central node.

In [ ]:
aggregator = shfl.federated_aggregator.AvgFedAggregator()
federated_government = shfl.learning_approach.FederatedGovernment(model_builder, federated_data, aggregator)

Before running the algorithm, we want to apply a transformation to the data. 
will ensure that the transformation is applied to the federated data in all the client nodes. 
We want to reshape the data, so we define the following FederatedTransformation.

In [ ]:
import numpy as np

class Reshape(shfl.private.FederatedTransformation):
    
    def apply(self, labeled_data):
        labeled_data.data = np.reshape(labeled_data.data, (labeled_data.data.shape[0], labeled_data.data.shape[1], labeled_data.data.shape[2],1))
        
shfl.private.federated_operation.apply_federated_transformation(federated_data, Reshape())

In addition, we want to normalize the data. 
We define a federated transformation using mean and standard deviation (std) estimates from the training set in this example. 

In [ ]:
import numpy as np

class Normalize(shfl.private.FederatedTransformation):
    
    def __init__(self, mean, std):
        self.__mean = mean
        self.__std = std
    
    def apply(self, labeled_data):
        labeled_data.data = (labeled_data.data - self.__mean)/self.__std
        
        
mean = np.mean(train_data.data)
std = np.std(train_data.data)
shfl.private.federated_operation.apply_federated_transformation(federated_data, Normalize(mean, std))

We are now ready to execute our federated learning algorithm.

In [ ]:
test_data = np.reshape(test_data, (test_data.shape[0], test_data.shape[1], test_data.shape[2],1))
federated_government.run_rounds(3, test_data, test_labels)